In [1]:
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,mean_squared_error,r2_score,mean_absolute_error
from sklearn.linear_model import Ridge

In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv('energydata_complete.csv')

In [6]:
df.drop(columns = ['date','lights'], inplace = True)

In [7]:
#X = df.drop(columns =['Appliances'])

In [8]:
#y = df['Appliances']

In [12]:
normalised_df = pd.DataFrame(scaler.fit_transform(df) , columns = df.columns)
normalised_df.head()
normalised_df = normalised_df.drop(columns = ['Appliances'])
normalised_df.head()

,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,T5,RH_5,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,0.32735,0.566187,0.225345,0.684038,0.215188,0.746066,0.351351,0.764262,0.175506,0.381691,...,0.223032,0.677290,0.372990,0.097674,0.894737,0.500000,0.953846,0.538462,0.265449,0.265449
1,0.32735,0.541326,0.225345,0.682140,0.215188,0.748871,0.351351,0.782437,0.175506,0.381691,...,0.226500,0.678532,0.369239,0.100000,0.894737,0.476190,0.894872,0.533937,0.372083,0.372083
2,0.32735,0.530502,0.225345,0.679445,0.215188,0.755569,0.344745,0.778062,0.175506,0.380037,...,0.219563,0.676049,0.365488,0.102326,0.894737,0.452381,0.835897,0.529412,0.572848,0.572848
3,0.32735,0.524080,0.225345,0.678414,0.215188,0.758685,0.341441,0.770949,0.175506,0.380037,...,0.219563,0.671909,0.361736,0.104651,0.894737,0.428571,0.776923,0.524887,0.908261,0.908261
4,0.32735,0.531419,0.225345,0.676727,0.215188,0.758685,0.341441,0.762697,0.178691,0.380037,...,0.219563,0.671909,0.357985,0.106977,0.894737,0.404762,0.717949,0.520362,0.201611,0.201611


In [10]:
target = normalised_df['Appliances']
target.head()

0    0.046729
1    0.046729
2    0.037383
3    0.037383
4    0.046729
Name: Appliances, dtype: float64

In [13]:
x_train,x_test,y_train,y_test = train_test_split(normalised_df,target,test_size= 0.3, random_state = 42)


In [14]:
regressor = LinearRegression()

In [15]:
regressor.fit(x_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [16]:
predicted_values = regressor.predict(x_test)

In [20]:
mae = mean_absolute_error(y_test,predicted_values)
round(mae,3)

0.05

In [64]:
rss = np.sum(np.square(y_test-predicted_values))

In [65]:
round(rss, 3)

45.348

In [66]:
mse = np.sqrt(mean_squared_error(y_test,predicted_values))
round(mse,3)

0.088

In [61]:
from sklearn.linear_model import Ridge
ridge_reg = Ridge(alpha=0.4)
ridge_reg.fit(x_train,y_train)
x=ridge_reg.predict(x_test)

In [53]:
from sklearn.linear_model import Lasso
lasso_reg = Lasso(alpha = 0.001)
lasso_reg = lasso_reg.fit(x_train,y_train)
l = lasso_reg.predict(x_test)


In [55]:
mse = np.sqrt(mean_squared_error(y_test,l))
round(mse,3)

0.094

In [70]:
r2_score = r2_score(y_test,l)

TypeError: 'numpy.float64' object is not callable

In [62]:
mse = np.sqrt(mean_squared_error(y_test,x))

In [63]:
round(mse,3)

0.088

In [43]:
#comparing the effects of regularisation
def get_weights_df(model, feat, col_name):
#this function returns the weight of every feature
    weights = pd.Series(model.coef_, feat.columns).sort_values()
    weights_df = pd.DataFrame(weights).reset_index()
    weights_df.columns = ['Features', col_name]
    weights_df[col_name].round(3)
    return weights_df

In [44]:
linear_model_weights = get_weights_df(regressor, x_train, 'Linear_Model_Weight')
ridge_weights_df = get_weights_df(ridge_reg, x_train, 'Ridge_Weight')
lasso_weights_df = get_weights_df(lasso_reg, x_train, 'Lasso_weight')

In [45]:
final_weights = pd.merge(linear_model_weights, ridge_weights_df, on='Features')
final_weights = pd.merge(final_weights, lasso_weights_df, on='Features')


In [46]:
final_weights

,Features,Linear_Model_Weight,Ridge_Weight,Lasso_weight
0,RH_2,-0.456698,-0.401134,-0.000000
1,T_out,-0.321860,-0.250765,0.000000
2,T2,-0.236178,-0.193880,0.000000
3,T9,-0.189941,-0.188584,-0.000000
4,RH_8,-0.157595,-0.156596,-0.000110
5,RH_out,-0.077671,-0.050541,-0.049557
6,RH_7,-0.044614,-0.046291,-0.000000
7,RH_9,-0.039800,-0.041701,-0.000000
8,T5,-0.015657,-0.020727,-0.000000
9,T1,-0.003281,-0.021549,0.000000


In [47]:
final_weights['Linear_Model_Weight'].max()

0.5535465998386386

In [71]:
final_weights['Linear_Model_Weight'].min()

-0.45669794833849997